According to https://www.youtube.com/watch?v=Xxxuw4_iCzw Using HuggingFaceHub with API

In [19]:
!pip -q install langchain huggingface_hub transformers sentence_transformers accelerate bitsandbytes

In [20]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'HUGGINGFACEHUB_API_TOKEN' # use your token to replace it

Using HuggingFaceHub via API


*   T5 Encoder-decoder model




In [21]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

template = """Question: {question}

Answer \: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"]) # speify the template and the question that is expecting

In [22]:
# define a chain
llm_chain = LLMChain(prompt=prompt,
                     llm=HuggingFaceHub(repo_id="google/flan-t5-xl", # https://huggingface.co/google/flan-t5-xl
                                        model_kwargs={"temperature":0, 
                                                      "max_length":64})) #specify the LLM that we use, in this case, HuggingFaceHub

Q&A

In [23]:
#question = "What is the capital of England?"

#print(llm_chain.run(question))


ValueError: ignored

Local Pipeline: download model locally
T5 Flan Encoder-decoder (Seq2seq model)

In [29]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

model_id = 'google/flan-t5-small'
tokenizer = AutoTokenizer.from_pretrained(model_id) # every moel has its own tokenizer. Make sure to define the tokenizer for the specific model.
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True, device_map='auto')

# define pipeline
pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=128
)

local_llm = HuggingFacePipeline(pipeline=pipeline)


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 7>:7                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:467 in          │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   464 │   │   │   )                                                                              │
│   465 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   466 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│ ❱ 467 │   │   │   return model_class.from_pretrained(                                            │
│   468 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   469 │   │   │   )                                                                              │
│   470 │   │   raise ValueError(                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2722 in from_pretrained   │
│                                                                                                  │
│   2719 │   │   │   │   │   key: device_map[key] for key in device_map.keys() if key not in modu  │
│   2720 │   │   │   │   }                                                                         │
│   2721 │   │   │   │   if "cpu" in device_map_without_lm_head.values() or "disk" in device_map_  │
│ ❱ 2722 │   │   │   │   │   raise ValueError(                                                     │
│   2723 │   │   │   │   │   │   """                                                               │
│   2724 │   │   │   │   │   │   Some modules are dispatched on the CPU or the disk. Make sure yo  │
│   2725 │   │   │   │   │   │   the quantized model. If you want to dispatch the model on the CP  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: 
                        Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to 
fit
                        the quantized model. If you want to dispatch the model on the CPU or the disk while keeping
                        these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a 
custom
                        `device_map` to `from_pretrained`. Check
                        https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-
cpu-and-gpu
                        for more details.

In [ ]:
# define a chain
llm_chain = LLMChain(prompt=prompt,
                     llm=local_llm) #specify the LLM that we use, in this case, the local LLM